<a href="https://colab.research.google.com/github/ikonushok/My_studying_Data-Science-and-Neyro-Nets-on-Python/blob/master/%D0%94%D0%97_2_%D0%9E%D0%B1%D1%83%D1%87%D0%B0%D1%8E%D1%89%D0%B0%D1%8F_%D0%B8_%D1%82%D0%B5%D1%81%D1%82%D0%BE%D0%B2%D0%B0%D1%8F_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8(%D0%9A%D0%BE%D0%BD%D1%8E%D1%88%D0%BE%D0%BA).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LIGHT
Создайте модель для распознавания рукописных цифр из набора MNIST (можно воспользоваться ноутбуком 1-го занятия) и проведите ряд тестов:

## Запустите сеть с различными размерами обучающей выборки:




In [ ]:
# Открываем необходимые библиотеки
from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import utils # для to_categorical
from tensorflow.keras.callbacks import EarlyStopping # для остановки при начале переобучения

import numpy as np
import pandas as pd

import pylab
import matplotlib.pyplot as plt
from  PIL import Image
# отрисовывать в ноутбуке
%matplotlib inline 

In [ ]:
# Загрузка данных
(x_train_org, y_train_org), (x_test_org, y_test_org) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


###Обучающие выборки: от 60.000 до 10 образцов
для сравнения оставил и первоначальную выборку в 60.000 образцов

In [ ]:
# необходимые переменные
N = [60000, 50000, 10000, 5000, 1000, 500, 250, 100, 50, 10] # Размеры тестовых выборок
N_test = 10000 # доля тестовой выборки от обучающей
N_batch_size = 128
N_Dence_relu = 800
N_epochs = 25 # от переобучения спасет EarlyStopping
data = [[0, 0, 800, 'relu', N_batch_size, 0]]

In [ ]:
for i in N:
  # берем нужную выборку
  x_train = x_train_org[0: i]
  y_train = y_train_org[0: i]
  # меняем формат картинок с 28х28 на 784х1
  x_train = x_train.reshape(i, 784)
  x_test = x_test_org.reshape(N_test, 784)  # тоже можно менять размер пропорционально N
  # нормализуем данные
  x_train = x_train.astype('float32')
  x_train = x_train / 255 # делим на 255 чтобы диапазон был от 0 до 1
  x_test = x_test.astype('float32')
  x_test = x_test / 255 # делим на 255 чтобы диапазон был от 0 до 1
  # преобразуем ответы в формат one_hot_encoding
  y_train = utils.to_categorical(y_train, 10)
  y_test = utils.to_categorical(y_test_org, 10)
  # проверяем размеры
  #print('x_train:' , x_train.shape, '\t'
  #      'y_train:' , y_train.shape, '\t'
  #      'x_test:' , x_test.shape, '\t'
  #      'y_test:' , y_test.shape, '\n')

  # создаем нейросеть
  model=Sequential() # нейросеть прямого распространения
  model.add(Dense(N_Dence_relu, input_dim=784, activation='relu')) # добавили полносвязный слой
  model.add(Dense(10, activation='softmax')) # полносвязный слой на выходе
  # компилируем
  model.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])

  # задание условий установки обучения
  early_stopping = EarlyStopping(monitor = 'val_loss',
                                 patience=1, # 1 мало, но существенно ускоряет процесс
                                 # при этом потеря в качестве очень мала
                                 # Число эпох, в течении которых должен ухудшаться val_loss
                                 restore_best_weights = True)

  # обучаем сеть
  history = model.fit(x_train, y_train,
                      batch_size=N_batch_size, 
                      epochs=N_epochs, 
                      verbose=0, # не показывать процесс обучения
                      validation_split=0.2,
                      callbacks=[early_stopping])

  # готовим отчетную таблицу
  data = data + [[x_train.shape, x_test.shape, N_Dence_relu, 'linear', N_batch_size, 
                  round(model.evaluate(x_test, y_test, verbose = 0)[1], 3)]]
  
  # проверяем результат обучения на тестовой выборке
  #print("Поверяем результат обучения для выборки: ", i)
  #model.evaluate(x_test, y_test)[1]

# Печатаем отчет
df = pd.DataFrame(data[1:], 
                  columns = ['train', 'test', 'neurons', 'activation', 'batch_size','val_accuracy'])
df


,train,test,neurons,activation,batch_size,val_accuracy
0,"(60000, 784)","(10000, 784)",800,linear,128,0.979
1,"(50000, 784)","(10000, 784)",800,linear,128,0.979
2,"(10000, 784)","(10000, 784)",800,linear,128,0.945
3,"(5000, 784)","(10000, 784)",800,linear,128,0.936
4,"(1000, 784)","(10000, 784)",800,linear,128,0.869
5,"(500, 784)","(10000, 784)",800,linear,128,0.791
6,"(250, 784)","(10000, 784)",800,linear,128,0.760
7,"(100, 784)","(10000, 784)",800,linear,128,0.676
8,"(50, 784)","(10000, 784)",800,linear,128,0.610
9,"(10, 784)","(10000, 784)",800,linear,128,0.369


In [ ]:
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_18 (Dense)             (None, 800)               628000    
_________________________________________________________________
dense_19 (Dense)             (None, 10)                8010      
Total params: 636,010
Trainable params: 636,010
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# код не запускать!
# Сделано без EarlyStopping

,samples,neurons,activation,batch_size,val_accuracy
0,60000,800,linear,128,0.982
1,50000,800,linear,128,0.980
2,10000,800,linear,128,0.956
3,500,800,linear,128,0.824


### Вывод:

1. Эксперимент с EarlyStopping показал, что 1 дает снижение качества обучения на 1%, но patience=3 практически не снижает качество обучения, заметно его ускоряя!
2. Эффективная выборка для обучения такой нейрости должна быть не менее 5.000 образцов
3. Было бы хорошо визуализировать падение val_accuracy в зависимости от размера выборки, но пока не разобрался как 


##Создайте сеть следующей архитектуры:
a. 4 Dense слоя

b. 3 Dropout слоя

c. 3 BatchNormalization слоя

In [ ]:
# необходимые переменные
N = [60000, 50000, 10000, 5000, 1000, 500, 250, 100, 50, 10] # Размеры тестовых выборок
N_test = 10000 # доля тестовой выборки от обучающей
N_batch_size = 128
N_Dence_relu = 800
N_epochs = 25 # от переобучения спасет EarlyStopping
data = [[0, 0, 800, 'relu', N_batch_size, 0]]

k_Dropout = 0.3 # доля отключаемых для обучения нейронов

In [ ]:
for i in N:
  # берем нужную выборку
  x_train = x_train_org[0: i]
  y_train = y_train_org[0: i]
  # меняем формат картинок с 28х28 на 784х1
  x_train = x_train.reshape(i, 784)
  x_test = x_test_org.reshape(N_test, 784)
  # нормализуем данные
  x_train = x_train.astype('float32')
  x_train = x_train / 255 # делим на 255 чтобы диапазон был от 0 до 1
  x_test = x_test.astype('float32')
  x_test = x_test / 255 # делим на 255 чтобы диапазон был от 0 до 1
  # преобразуем ответы в формат one_hot_encoding
  y_train = utils.to_categorical(y_train, 10)
  y_test = utils.to_categorical(y_test_org, 10)

  # создаем нейросеть
  model2=Sequential() # нейросеть прямого распространения
  #1
  model2.add(Dropout(k_Dropout))
  model2.add(Dense(N_Dence_relu, input_dim=784, activation='relu'))
  #2
  model2.add(Dropout(k_Dropout))
  model2.add(BatchNormalization())
  model2.add(Dense(N_Dence_relu/2, input_dim=784, activation='relu')) # число нейронов/2
  #3
  model2.add(Dropout(k_Dropout))
  model2.add(BatchNormalization())
  model2.add(Dense(N_Dence_relu/4, input_dim=784, activation='relu')) # число нейронов/4
  #4 
  model2.add(BatchNormalization())
  model2.add(Dense(10, activation='softmax')) # полносвязный слой на выходе
  # компилируем
  model2.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])
  
  # задание условий установки обучения
  early_stopping = EarlyStopping(monitor = 'val_loss',
                                 patience=1, # лучше 3
                                 restore_best_weights = True)
  # обучаем сеть
  model2.fit(x_train, y_train, 
            batch_size=N_batch_size, 
            epochs=N_epochs, 
            verbose=0, # не показывать процесс обучения
            validation_split=0.2,
            callbacks=[early_stopping])
  
  # готовим отчетную таблицу
  data = data + [[x_train.shape, x_test.shape, N_Dence_relu, 'linear', N_batch_size, 
                  round(model2.evaluate(x_test, y_test, verbose = 0)[1], 3)]]


# Печатаем отчет
df = pd.DataFrame(data[1:], 
                  columns = ['train', 'test', 'neurons', 'activation', 'batch_size','val_accuracy'])
df

,train,test,neurons,activation,batch_size,val_accuracy
0,"(60000, 784)","(10000, 784)",800,linear,128,0.979
1,"(50000, 784)","(10000, 784)",800,linear,128,0.973
2,"(10000, 784)","(10000, 784)",800,linear,128,0.953
3,"(5000, 784)","(10000, 784)",800,linear,128,0.941
4,"(1000, 784)","(10000, 784)",800,linear,128,0.873
5,"(500, 784)","(10000, 784)",800,linear,128,0.816
6,"(250, 784)","(10000, 784)",800,linear,128,0.729
7,"(100, 784)","(10000, 784)",800,linear,128,0.574
8,"(50, 784)","(10000, 784)",800,linear,128,0.403
9,"(10, 784)","(10000, 784)",800,linear,128,0.415


In [ ]:
print(model2.summary())

Model: "sequential_126"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_201 (Dropout)        (None, 784)               0         
_________________________________________________________________
dense_386 (Dense)            (None, 800)               628000    
_________________________________________________________________
dropout_202 (Dropout)        (None, 800)               0         
_________________________________________________________________
batch_normalization_201 (Bat (None, 800)               3200      
_________________________________________________________________
dense_387 (Dense)            (None, 400)               320400    
_________________________________________________________________
dropout_203 (Dropout)        (None, 400)               0         
_________________________________________________________________
batch_normalization_202 (Bat (None, 400)            

------
###Выводы по результатам проведенных тестов.
1. BatchNormalization в данном случае вообще не нужен
2. Думал постараться загнать создание нейросети в функцию, но, похоже, тут тоже не будет выигрыша
3. Dropout может быть полезен. Особенно при небольих выборках

# PRO

##Вариант 1
Повысьте точность модели по обнаружению мин до 90 % на тестовой выборке. Можно использовать различные варианты слоев Dropout и BatchNormalization. Можно менять количество примеров в обучающей и проверочной выборках, но нельзя менять количество примеров в тестовой.

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data Science и нейронные сети на Python/Занятие_2_Обучающая и тестовая выборки/base2/sonar.csv", header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [ ]:
print(df.shape)

(208, 61)


In [ ]:
# готовим данные для нейросети
dataset = df.values # берем значения без индексов
X = dataset[:, :-1].astype(float) # присваиваем float данным с эхолота
Y = dataset[:,-1] # выделяем последний столбец в Y
Y[Y=='R'] = '0'   # 0 - для всех, кроме мин
Y[Y=='M'] = '1'   # 1 - для мин
Y = Y.astype(int)   # переводим Y в int
print(X.shape , Y.shape)
print(Y)

(208, 60) (208,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [ ]:
# разделяем данные на тестовую и обучающую выборки
from sklearn.model_selection import train_test_split

# X, Y - разделяемые выборки
# test_size=0.2 - 20% не проверочную выборку
# shuffle=True - перемешать данные
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

# проверяем размерности
print('X: ', x_train.shape, x_test.shape, '\t Y: ',  y_train.shape, y_test.shape)

X:  (166, 60) (42, 60) 	 Y:  (166,) (42,)


### Эталонная сеть:

In [ ]:
from tensorflow.keras.optimizers import Adam # алгоритм для настройки скорости обучения сети

# функция, создающая нейронную сеть
def createModel():
  model = Sequential()
  model.add(Dense(60, input_dim=60, activation='relu'))
  model.add(Dense(30, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # Компилируем сеть
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

  return model

In [ ]:
# создаем пустую сеть и обучаем ее
model = createModel()

model.fit(x_train, y_train,
          batch_size=8,
          epochs=100,
          verbose=0)

scores_train = model.evaluate(x_train, y_train, verbose=1)
print('Результаты на обучающей выборке: ', scores_train)
scores_test = model.evaluate(x_test, y_test, verbose=1)
print('Результаты на тестовой выборке: ',scores_test)

6/6 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 1.0000
Результаты на обучающей выборке:  [0.014162193052470684, 1.0]
2/2 [==============================] - 0s 3ms/step - loss: 0.1857 - accuracy: 0.9048
Результаты на тестовой выборке:  [0.1856997311115265, 0.9047619104385376]


### повышаем точность до 90%

In [ ]:
# создаем нейронную сеть
# ПОМНИМ О ТОМ, ЧТО ЧЕМ МЕНЬШЕ ВЫБОРКА - ТЕМ ПРОЩЕ СЕТЬ!!
def createModel2(n):
  model2 = Sequential()
  model2.add(Dense(n, input_dim=60, activation='relu'))
  model2.add(Dense(1, activation='sigmoid'))
  # Компилируем сеть (тут я ускорил в 5 раз скорость обучения)
  model2.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.005), metrics=['accuracy'])

  return model2

In [ ]:
# создаем пустую сеть и обучаем ее в 5-ти тестовых прогонах
n = [120]
for j in n:
  print('\n Число нейронов: ', j)
  for i in range(5):
    model2 = createModel2(j)
    # задание условий установки обучения после 5-го ухудшения 'loss'
    early_stopping = EarlyStopping(monitor ='loss', patience=5, restore_best_weights = True)
    model2.fit(x_train, y_train, batch_size=12, epochs=200, verbose=0, callbacks=[early_stopping])

    #print('Результаты на обучающей выборке: ', model2.evaluate(x_train, y_train, verbose=1))
    score_test = model2.evaluate(x_test, y_test, verbose=1)
    print('Результаты на тестовой выборке: ', score_test)


 Число нейронов:  120
2/2 [==============================] - 0s 5ms/step - loss: 0.1494 - accuracy: 0.9048
Результаты на тестовой выборке:  [0.14944487810134888, 0.9047619104385376]
2/2 [==============================] - 0s 7ms/step - loss: 0.1144 - accuracy: 0.9762
Результаты на тестовой выборке:  [0.11436201632022858, 0.976190447807312]
2/2 [==============================] - 0s 5ms/step - loss: 0.1811 - accuracy: 0.9048
Результаты на тестовой выборке:  [0.18107812106609344, 0.9047619104385376]
2/2 [==============================] - 0s 3ms/step - loss: 0.1011 - accuracy: 0.9762
Результаты на тестовой выборке:  [0.10108629614114761, 0.976190447807312]
2/2 [==============================] - 0s 5ms/step - loss: 0.1074 - accuracy: 0.9286
Результаты на тестовой выборке:  [0.10739497095346451, 0.9285714030265808]


### Вывод:
1. исходил из того, что чем меньше примеров, тем проще должна быть сеть
2. поставил большое количество эпох и автоматическую остановку при ухудшении loss после 5-й эпохи подряд
3. после экспериментов с числом нейронов, увеличил их до 120
4. в 5 раз ускорил  обучение (думаю, тем самым немного ушел от раннего переобучения)
5. с размером выборки решил не экпериментировать, тк он и там чрезвычайно мал
6. для надежности делал по 5 прогонов